# Prompt Engineering

Prompt engineering is a powerful technique that can help to optimize language models for various applications and research topics. By creating good prompts, the model can be guided to deliver accurate, contextually relevant, and insightful responses.

* [1. Intro to Prompting](#intro)
    * [1.1. Role Prompting](#role)
    * [1.2. Few Shot Prompting](#few_shot)
    * [1.3. Bad Prompt Practices](#bad) 
    * [1.4. Chain Prompting](#chain)
    * [1.5. Well-Structured Prompt Example](#well)
* [2. Using Prompt Templates](#templates)
    * [2.1. PromptTemplate](#prompttemp)
    * [2.2. FewShotPromptTemplate](#fewshottemp)
    * [2.3. Saving PromptTemplate and Loading it back](#savetemp)
    * [2.4. Example: Teach LLM to Respond Sarcastically](#sarcastic)
    * [2.5. LengthBasedExampleSelector](#example_selector)
    * [2.6. Alternating Human/AI messages](#human_ai)
    * [2.7. SemanticSimilarityExampleSelector](#semantic)
* [3. Additional Resources](#resources)

In [1]:
import os
from keys import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

<hr>
<a class="anchor" id="intro">
    
## 1. Intro to Prompting
    
</a>

<hr>
<a class="anchor" id="role">
    
### 1.1 Role Prompting
    
</a>

Role prompting involves asking the LLM to assume a specific role or identity before performing a given task, such as acting as a copywriter. This can help guide the model's response by providing a context or perspective for the task. 

In [2]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?
"""
prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template,
)

# Create the LLMChain for the prompt
llm = OpenAI(model_name="text-davinci-003", temperature=0)

# Create LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [3]:
# Input data for the prompt
input_data = {"theme": "interstellar travel", "year": "3030"} 

In [4]:
# Run the LLMChain to get the AI-generated song title
response = chain.run(input_data)

print("Theme:", input_data["theme"])
print("Year:", input_data["year"])
print("AI-generated song title:", response)

Theme: interstellar travel
Year: 3030
AI-generated song title: 
"Journey to the Stars: 3030"


The provided prompt is a good example of role prompting  for several reasons:

- Clear instructions
- Specificity
- Open-ended creativity
- Focus on the task

<hr>
<a class="anchor" id="few_shot">
    
### 1.2. Few Shot Prompting
    
</a>

In [5]:
from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)

In [6]:
examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""

example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=["input"],
    example_separator="\n",
)

In [7]:
formatted_prompt = few_shot_prompt.format(input="purple")

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=PromptTemplate(template=formatted_prompt, input_variables=[]))

# Run the LLMChain to get the AI-generated emotion associated with the input color
response = chain.run({})

print("Color: purple")
print("Emotion:", response)

Color: purple
Emotion:  creativity


<hr>
<a class="anchor" id="bad">
    
### 1.3. Bad Prompt Practices
    
</a>

In [23]:
# An example of a too-vague prompt that provides little context or guidance 
# for the model to generate a meaningful response

from langchain import PromptTemplate

template = "Tell me something about {topic}."
prompt = PromptTemplate(
    input_variables=["topic"],
    template=template,
)
prompt.format(topic="Europe")

'Tell me something about Europe.'

<hr>
<a class="anchor" id="chain">
    
### 1.4. Chain Prompting
    
</a>

Chain Prompting refers to the practice of chaining consecutive prompts, where the output of a previous prompt becomes the input of the successive prompt.

In [24]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

In [25]:
# Prompt 1
template_question = """What is the name of the famous scientist who developed the theory of general relativity?
Answer: """
prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """Provide a brief description of {scientist}'s theory of general relativity.
Answer: """
prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

In [26]:
# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0) 

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response
scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)

Scientist: Albert Einstein
Fact: 
Albert Einstein's theory of general relativity is a theory of gravitation that states that the gravitational force between two objects is a result of the curvature of spacetime caused by the presence of mass and energy. It explains the phenomenon of gravity as a result of the warping of space and time by matter and energy.


<hr>
<a class="anchor" id="well">
    
### 1.5. Well-Structured Prompt Example
    
</a>

Tips to engineer effective prompts:
- Be specific (provide detail and enough context to guide the LLM toward the desired output);
- Force conciseness when needed;
- Encourage the model to explain its reasoning (this way the results can be more accurate, especially for complex tasks);
- Iterate if needed (several refinements may be required to obtain the best possible answer).

In [28]:
from langchain import FewShotPromptTemplate, PromptTemplate, LLMChain
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="text-davinci-003", temperature=0)

examples = [
    {
        "query": "What's the secret to happiness?",
        "answer": "Finding balance in life and learning to enjoy the small moments."
    }, {
        "query": "How can I become more productive?",
        "answer": "Try prioritizing tasks, setting goals, and maintaining a healthy work-life balance."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
life coach. The assistant provides insightful and practical advice to the users' questions. Here are some
examples: 
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the LLMChain for the few-shot prompt template
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

# Define the user query
user_query = "What are some tips for improving communication skills?"

# Run the LLMChain for the user query
response = chain.run({"query": user_query})

print("User Query:", user_query)
print("AI Response:", response)

User Query: What are some tips for improving communication skills?
AI Response:  Practice active listening, be mindful of your body language, and be open to constructive feedback.


<hr>
<a class="anchor" id="templates">
    
## 2. Using Prompt Templates
    
</a>

<a class="anchor" id="prompttemp">
    
### 2.1. PromptTemplate
    
</a>

A PromptTemplate is a predefined structure or pattern used to construct effective and consistent prompts for large language models. It is a guideline to ensure the input text or prompt is properly formatted.

To create a `PromptTemplate` object, two arguments are required:

1. `input_variables`: A list of variable names in the template;
2. `template`: The template string containing formatted text and placeholders.

In [29]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided, answer
with "I don't know".
Context: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.
...
Question: {query}
Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=prompt_template)

# Set the query you want to ask
input_data = {"query": "What is the main advantage of quantum computing over classical computing?"}

# Run the LLMChain to get the AI-generated answer
response = chain.run(input_data)

print("Question:", input_data["query"])
print("Answer:", response)

Question: What is the main advantage of quantum computing over classical computing?
Answer:  The main advantage of quantum computing over classical computing is its ability to solve complex problems faster.


<a class="anchor" id="fewshottemp">
    
### 2.2. FewShotPromptTemplate
    
</a>

In [30]:
# For more advanced usage, it is possible to create a FewShotPromptTemplate 
# with an ExampleSelector to select a subset of examples that will be most informative for the language model.

from langchain import LLMChain, FewShotPromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"}
]

example_template = """
Animal: {animal}
Habitat: {habitat}
"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template=example_template
)

dynamic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal",
    suffix="Animal: {input}\nHabitat:",
    input_variables=["input"],
    example_separator="\n\n",
)

# Create the LLMChain for the dynamic_prompt
chain = LLMChain(llm=llm, prompt=dynamic_prompt)

# Run the LLMChain with input_data
input_data = {"input": "tiger"}
response = chain.run(input_data)

print(response)

 tropical forests and mangrove swamps


<a class="anchor" id="savetemp">
    
### 2.3.  Saving PromptTemplate and Loading it back
    
</a>

In [31]:
# Save PromptTemplate to a JSON file (or YAML file)
prompt_template.save("output/my_prompt.json")

In [32]:
# Loading back
from langchain.prompts import load_prompt

loaded_prompt = load_prompt("output/my_prompt.json")
print(loaded_prompt)

input_variables=['query'] output_parser=None partial_variables={} template='Answer the question based on the context below. If the\nquestion cannot be answered using the information provided, answer\nwith "I don\'t know".\nContext: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.\n...\nQuestion: {query}\nAnswer: ' template_format='f-string' validate_template=True


<a class="anchor" id="sarcastic">
    
### 2.4. Example: Teach LLM to Respond Sarcastically
    
</a>

In [33]:
# Teach the LLM by providing examples to respond sarcastically to questions
from langchain import LLMChain, FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

examples = [
    {
        "query": "How do I become a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    }, {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny 
responses to users' questions. Here are some examples: 
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the LLMChain for the few_shot_prompt_template
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

# Run the LLMChain with input_data
input_data = {"query": "How can I learn natural language processing?"}
response = chain.run(input_data)

print(response)

 Start by learning the language of the birds. Once you've mastered that, you'll be well on your way to mastering natural language processing.


<a class="anchor" id="example_selector">
    
### 2.5. LengthBasedExampleSelector
    
</a>

It is essential to optimize the performance of few-shot learning, providing the model with as many relevant examples as possible while staying within the limits of the maximum context window and avoiding excessive processing times. The dynamic inclusion or exclusion of examples allows us to achieve a balance between providing sufficient context and maintaining efficiency in the model's operation:

In [37]:
examples = [
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    }, {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    }, {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    }, {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    }, {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    }, {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    }, {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    }, {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    }, {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    }, {
        "query": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world."
    }, {
        "query": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom."
    }, {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    }, {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    }
]

In [38]:
# Instead of utilizing the examples list of dictionaries directly, let's implement a LengthBasedExampleSelector
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  
)

In [39]:
# To dynamically select and include examples based on their length, 
# (ensuring that the final prompt stays within the desired token limit)

dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector, 
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

In [43]:
from langchain import LLMChain, FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI
from langchain.prompts.example_selector import LengthBasedExampleSelector

llm = OpenAI(model_name="gpt-3.5-turbo")

# Create the LLMChain for the dynamic_prompt_template
chain = LLMChain(llm=llm, prompt=dynamic_prompt_template)

# Run the LLMChain with input_data
input_data = {"query": "Who invented the telephone?"}
response = chain.run(input_data)

print(response)

Alexander Graham Bell, the man who decided we needed a really loud and obnoxious way for our friends to interrupt us.


In [45]:
# Another example of using LengthBasedExampleSelector

from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

example_template = """
Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template
)

# Create an instance of LengthBasedExampleSelector
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,
)

# Create a FewShotPromptTemplate
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n",
)

print(dynamic_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad



Word: tall
Antonym: short



Word: energetic
Antonym: lethargic



Word: sunny
Antonym: gloomy


Word: big
Antonym:


In [47]:
chain = LLMChain(llm=llm, prompt=dynamic_prompt)
response = chain.run(input="big")
print(response)

small


<a class="anchor" id="human_ai">
    
### 2.6. Alternating Human/AI messages
    
</a>

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template="You are a helpful assistant that translates english to pirate."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, 
                                                example_human, 
                                                example_ai, 
                                                human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run("I love programming.")

"I be lovin' the art of code plunderin'."

<a class="anchor" id="semantic">
    
### 2.7. SemanticSimilarityExampleSelector
    
</a>

In [48]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Create a PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Define some examples
examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

In [50]:
# Embedding function
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [52]:
# Create Deep Lake dataset

from keys import ACTIVELOOP_TOKEN
os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN

my_activeloop_org_id = "iryna" 
my_activeloop_dataset_name = "langchain_course_fewshot_selector"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


In [56]:
# Instantiate SemanticSimilarityExampleSelector using the examples
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, embeddings, db, k=1
)

# Create a FewShotPromptTemplate using the example_selector
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Convert the temperature from Celsius to Fahrenheit",
    suffix="Input: {temperature}\nOutput:", 
    input_variables=["temperature"],
)

# Test the similar_prompt with different inputs
print(similar_prompt.format(temperature="10°C"))  # Test with an input
print(similar_prompt.format(temperature="30°C"))  # Test with another input

Deep Lake Dataset in ./deeplake/ already exists, loading from the storage


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.23s/it]


Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (21, 1536)  float32   None   
    id        text      (21, 1)      str     None   
 metadata     json      (21, 1)      str     None   
   text       text      (21, 1)      str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 10°C
Output: 50°F

Input: 10°C
Output:
Convert the temperature from Celsius to Fahrenheit

Input: 30°C
Output: 86°F

Input: 30°C
Output:


In [57]:
# Add a new example to the SemanticSimilarityExampleSelector
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(similar_prompt.format(temperature="40°C")) # Test with a new input after adding the example

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.86it/s]


Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (22, 1536)  float32   None   
    id        text      (22, 1)      str     None   
 metadata     json      (22, 1)      str     None   
   text       text      (22, 1)      str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 40°C
Output: 104°F

Input: 40°C
Output:


<hr>
<a class="anchor" id="resources">
    
## 3. Adiitional Resources:
    
</a>

- [A Hands-on Guide to Prompt Engineering with ChatGPT and GPT-3](https://dev.to/mmz001/a-hands-on-guide-to-prompt-engineering-with-chatgpt-and-gpt-3-4127)
- [Prompt Engineering Tips and Tricks with GPT-3](https://blog.andrewcantino.com/blog/2021/04/21/prompt-engineering-tips-and-tricks/)
- [Prompt Engineering LLMs with LangChain and W&B](https://wandb.ai/a-sh0ts/langchain_callback_demo/reports/Prompt-Engineering-LLMs-with-LangChain-and-W-B--VmlldzozNjk1NTUw)
- [Few-Shot Prompting](https://www.promptingguide.ai/techniques/fewshot)
- [ChatGPT Prompt Engineering Tips: Zero, One and Few Shot Prompting](https://www.allabtai.com/prompt-engineering-tips-zero-one-and-few-shot-prompting/)